# **Analysis Sentiment - Naive Bayes**

 

Sekarang kita akan mencoba untuk melakukan Analsis sentiment menggunakan salah satu metode Supervised Learning yaitu Naive Bayes. Seperti yang kita tahu bahwa pada supervised learning kita membutuhkan dataset agar mesin kita dapat belajar jadi pastika kita mempunyai dataset sudah memiliki dataset tersebut. Bisa dengan cara melakukan analisis sentiment terlebih dahulu dengan menggunakan perbandingan antara kata negatif-positif seperti kemarin, atau dengan mencari dataset yang sudah diberi label di internet.

Pada kesempatan ini kalian bisa menggunakan dataset tweet di bawah, untuk memperingan kinerja pada kesempatan pertama ini maka diberikan dataset dengan ukuran kecil

ini untuk dataset tweet yang diguakan

[dataset_tweet_2](https://blog.sanbercode.com/wp-content/uploads/2020/09/dataset_tweet_2.csv)

Mari kita coba baca dataset kita lalu membaginya kedalam dua atribut, yaitu tweet dan label.

In [ ]:
import pandas as pd
import csv

file = 'dataset_tweet_2.csv'

token_data = open(file)
tokens = csv.reader(token_data, delimiter=';')
tweets = []
label = []
for row in tokens:
    tweets.append(row[0])
    label.append(int(row[1].replace(',','')))

df = pd.DataFrame(columns=['tweets','label'])
df['tweets'] = tweets
df['label'] = label

print (df)

                                               tweets  label
0   rt @napqilla: no 1, 3 ambisinya menguasai raky...      1
1   rt @pandji: nah gue pikir sentimen petahana ok...      1
2   rt @pandji: urutan pertama best moment #debat2...      1
3   rt @pandji: ini artikel yg menjelaskan ternyat...      1
4   rt @mrtampi: agus makin santai.\nahok makin sa...      0
..                                                ...    ...
76  rt @pandji: nah gue pikir sentimen petahana ok...      0
77  rt @josua_tm: ibu sylvi adalah contoh bahwa wa...      1
78  besok saya ajak kesana saja, saya udah survei ...      1
79  benerr bgt.. dan tidak mengajak penonton ikut ...      1
80  rt @gandy_koz: pak anis,kl pas libur lebaran i...      1

[81 rows x 2 columns]


lalu selanjutnya kita akan melakukan pembersihan tweet di atas, kita akan memanfaatkan modul stemming pada sastrawi, jadi jangan lupa untuk menambahkan import pada bagian library. pertama kita akan melakukan case folding lalu kita akan melakukan stemming.

In [ ]:
pip install sastrawi

     |████████████████████████████████| 215kB 11.2MB/s 


In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory


import re,string

clean_tweets = []
for tweet in tweets:
    def hapus_tanda(tweet): 
        tanda_baca = set(string.punctuation)
        tweet = ''.join(ch for ch in tweet if ch not in tanda_baca)
        return tweet
    
    tweet=tweet.lower()
    tweet = re.sub(r'\\u\w\w\w\w', '', tweet)
    tweet=re.sub(r'http\S+','',tweet)
    #hapus @username
    tweet=re.sub('@[^\s]+','',tweet)
    #hapus #tagger 
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #hapus tanda baca
    tweet=hapus_tanda(tweet)
    #hapus angka dan angka yang berada dalam string 
    tweet=re.sub(r'\w*\d\w*', '',tweet).strip()
    
    #stemming
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    tweet = stemmer.stem(tweet)
    clean_tweets.append(tweet)

df['clean'] = clean_tweets
print(df.head())

                                              tweets  ...                                              clean
0  rt @napqilla: no 1, 3 ambisinya menguasai raky...  ...  rt no ambisi kuasa rakyat ambisi layan rakyat ...
1  rt @pandji: nah gue pikir sentimen petahana ok...  ...  rt nah gue pikir sentimen tahana oke di malam ...
2  rt @pandji: urutan pertama best moment #debat2...  ...  rt urut pertama best moment pak basuki misahin...
3  rt @pandji: ini artikel yg menjelaskan ternyat...  ...  rt ini artikel yg jelas nyata di yg dapet resp...
4  rt @mrtampi: agus makin santai.\nahok makin sa...  ...  rt agus makin santainahok makin santunnanies m...

[5 rows x 3 columns]


# **Machine Learning & Data Teks**

Seperti yang kita tahu untuk dapat menggunakan sebuah model machine learning maka kita membutuhkan data dengan tipe numerik atau integer, akan tetapi salah satu satu feature yang kita miliki merupakan data teks. Nah pada kesempatan ini kita akan memanfaatkan metode perubahan data teks - matriks yang sudah kita pelajari sebelumnya, yaitu TF-IDF

Mari kita mulai, kita akan memanfaatkan metode TfidVectorizer pada library sklearn dan gaussian Naive Bayes.



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB

vectorizer = TfidfVectorizer (max_features=2500)
model_g = GaussianNB()

Lalu kita ubah data clean_tweet kita ke dalam bentuk TFIDF Vectorizer

In [ ]:
v_data = vectorizer.fit_transform(df['clean']).toarray()

print(v_data)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(v_data, df['label'], test_size=0.2, random_state=0)
model_g.fit(X_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_preds = model_g.predict(X_test)

print(confusion_matrix(y_test,y_preds))
print(classification_report(y_test,y_preds))
print('nilai akurasinya adalah ',accuracy_score(y_test, y_preds))

[[8 4]
 [1 4]]
              precision    recall  f1-score   support

           0       0.89      0.67      0.76        12
           1       0.50      0.80      0.62         5

    accuracy                           0.71        17
   macro avg       0.69      0.73      0.69        17
weighted avg       0.77      0.71      0.72        17

nilai akurasinya adalah  0.7058823529411765


Untuk melakukan prediksi jangan lupa untuk mengubah 0 dan 1 menjadi sebuah parameter string pada akhir langkah.

In [ ]:
y_preds

array([1])

In [ ]:
tweet = ''
v_data = vectorizer.transform([tweet]).toarray()
y_preds = model_g.predict(v_data)

# dengan asumsi bahwa 1 merupan label positif
if y_preds == 1:
  print('positif')
else:
  print('negatif')

positif
